In [45]:
import json, nltk
nltk.download('punkt')
from langdetect import detect
import stanza 
from nltk.parse import DependencyGraph
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yutam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [93]:
from tqdm import tqdm

In [46]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [47]:
with open('annakarenina.txt', 'r', encoding='utf-8') as ak:
    text = ak.read()

In [57]:
mytext = '''Анна любила Вронского, но он не давал ей надежды на счастливое будущее.'''

In [58]:
doc = Doc(mytext)

In [59]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for token in doc.tokens:
    token.lemmatize(morph_vocab)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [60]:
new_text = ''
last = 0
for span in doc.ner.spans:
    if span.type == 'PER':
        name = ''
        start, stop = span.start, span.stop
        name =  mytext[start:stop]
        ana = morph.parse(name)
        ana[0].tag.gender
        name = name.replace(' ', '_')
        new_text += mytext[last:start] + name + '_PER' + '_' + ana[0].tag.gender
        last = span.stop
new_text += mytext[last:]

In [61]:
new_text

'Анна_PER_femn любила Вронского_PER_masc, но он не давал ей надежды на счастливое будущее.'

In [62]:
new_new = []
from string import punctuation
my_tokens = []
for t in new_text.split():
    if t[-1] in punctuation:
        my_tokens.append(t[:-1])
        my_tokens.append(t[-1])
    else:
        my_tokens.append(t)
for numb, token in enumerate(my_tokens):
    if morph.parse(token)[0].tag.POS == 'NPRO':
      #  print(token, morph.parse(token)[0].tag.gender)
        my_tokens[numb] = 'PRON_' + morph.parse(token)[0].tag.gender + '_' + morph.parse(token)[0].tag.case
for r in enumerate(list(reversed(my_tokens))):
   # print(r)
    if 'PRON_masc' in r[1]:
        for potential_name in list(reversed(my_tokens))[r[0]:]:
            if potential_name.endswith('masc'):
                new_new.append(potential_name + r[1][-5:])
                break
    elif 'PRON_femn' in r[1]:
        for potential_name in list(reversed(my_tokens))[r[0]:]:
            if potential_name.endswith('femn'):
                new_new.append(potential_name + r[1][-5:])
                break
    else:
        new_new.append(r[1])

In [68]:
new_new

['.',
 'будущее',
 'счастливое',
 'на',
 'надежды',
 'Анна_PER_femn_datv',
 'давал',
 'не',
 'Вронского_PER_masc_nomn',
 'но',
 ',',
 'Вронского_PER_masc',
 'любила',
 'Анна_PER_femn']

In [80]:
last_text = []
for word in new_new:
    if word.endswith('femn') or word.endswith('masc'):
        w = word.split('_')
        last_text.append(' '.join(w[:-2]))
    elif 'Кити' in word:
        last_text.append('Кити')
    elif 'Долли' in word:
        last_text.append('Долли')
    elif len(word) > 6 and word[-5] == '_':
        w = word.split('_')
        name = ' '.join(w[:-3])
        case = w[-1]
        n = morph.parse(name)[0]
        f = n.inflect({case})
        last_text.append(f.word.title())
    else:
        last_text.append(word)

In [81]:
last_text

['.',
 'будущее',
 'счастливое',
 'на',
 'надежды',
 'Анне',
 'давал',
 'не',
 'Вронский',
 'но',
 ',',
 'Вронского',
 'любила',
 'Анна']

In [82]:
words = ''
for a in list(reversed(last_text)):
    if a in [',', '.', ':', ';', '!', '?']:
        words += a
    else:
        words += ' ' + a

In [83]:
words.strip()

'Анна любила Вронского, но Вронский не давал Анне надежды на счастливое будущее.'

In [56]:
doc1 = Doc(text)
doc1.segment(segmenter)
doc1.tag_morph(morph_tagger)
for token in doc1.tokens:
    token.lemmatize(morph_vocab)
doc1.parse_syntax(syntax_parser)
doc1.tag_ner(ner_tagger)

KeyboardInterrupt: 

In [123]:
def remake_paragraph(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    new_text = ''
    last = 0
    for span in doc.ner.spans:
        if span.type == 'PER':
            name = ''
            start, stop = span.start, span.stop
            name =  text[start:stop]
            ana = morph.parse(name)
            if ana[0].tag.gender != None:
                name = name.replace(' ', '_')
                new_text += text[last:start] + name + '_PER' + '_' + ana[0].tag.gender
                last = span.stop
    new_text += text[last:]
    new_new = []
    from string import punctuation
    my_tokens = []
    for t in new_text.split():
        if t[-1] in punctuation:
            my_tokens.append(t[:-1])
            my_tokens.append(t[-1])
        else:
            my_tokens.append(t)
    for numb, token in enumerate(my_tokens):
        if morph.parse(token)[0].tag.POS == 'NPRO':
            if morph.parse(token)[0].tag.gender in ['masc', 'femn'] and morph.parse(token)[0].tag.case != None:
                my_tokens[numb] = 'PRON_' + morph.parse(token)[0].tag.gender + '_' + morph.parse(token)[0].tag.case
    for r in enumerate(list(reversed(my_tokens))):
        if 'PRON_masc' in r[1]:
            for potential_name in list(reversed(my_tokens))[r[0]:]:
                if potential_name.endswith('masc'):
                    new_new.append(potential_name + r[1][-5:])
                    break
        elif 'PRON_femn' in r[1]:
            for potential_name in list(reversed(my_tokens))[r[0]:]:
                if potential_name.endswith('femn'):
                    new_new.append(potential_name + r[1][-5:])
                    break
        else:
            new_new.append(r[1])
    last_text = []
   # print(new_new)
    for word in new_new:
        if word.endswith('femn') or word.endswith('masc'):
            w = word.split('_')
            last_text.append(' '.join(w[:-2]))
        elif 'Кити' in word:
            last_text.append('Кити')
        elif 'Долли' in word:
            last_text.append('Долли')
        elif len(word) > 6 and word[-5] == '_':
            w = word.split('_')
            name = ' '.join(w[:-3])
            case = w[-1]
            n = morph.parse(name)[0]
            f = n.inflect({case})
            if f != None:
                last_text.append(f.word.title())
            else:
                last_text.append(w[0])
        else:
            last_text.append(word)
    words = ''
    for a in list(reversed(last_text)):
        if a in [',', '.', ':', ';', '!', '?']:
            words += a
        else:
            words += ' ' + a
    return words.strip()

In [124]:
newbook = ''

In [125]:
for t in tqdm(paragraphs):
    if len(t) > 3:
       # print(remake_paragraph(t))
        newbook += remake_paragraph(t) + '\n\n'

100%|██████████████████████████████████████████████████████████████████████████████| 7807/7807 [06:21<00:00, 20.44it/s]


In [126]:
with open('new_ak.txt', 'w', encoding='utf-8') as file:
    file.write(newbook)